In [ ]:
%matplotlib tk
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from callbacks import ModelTest
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.models import Sequential, Input
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers import Concatenate
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l2
import keras.metrics
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
import keras.backend as K
from theano.sandbox.rng_mrg import MRG_RandomStreams


In [ ]:
from keras.datasets import mnist

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train.shape

In [ ]:
p_dropout = 0.5
weight_decay = 0.01
batch_size=400

In [ ]:
# i=np.random.randint(0, len(y_train))
# plt.imshow(X_train[i], cmap=plt.cm.get_cmap('Greys'))
# plt.title(y_train[i])
# plt.show()

In [ ]:
X_train = X_train.reshape(len(X_train), -1)
X_test = X_test.reshape(len(X_test), -1)

In [ ]:
#regression
# mean_y_train = np.mean(y_train)
# std_y_train = np.std(y_train)
# y_train = (y_train - mean_y_train) / std_y_train


In [ ]:
#classification
mean_y_train = 0
std_y_train = 1
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
X_train.shape, X_train.min(), X_train.max()

In [ ]:
from bayesian import BayesianDropoutModel, variation_ratio

In [ ]:
theano_rng = MRG_RandomStreams(42)


In [ ]:
# original loss from https://arxiv.org/pdf/1703.04977.pdf 
def bayesian_categorical_crossentropy_original(T, num_classes):
    def bayesian_categorical_crossentropy_internal_original(true, pred_var):
        # shape: (N,)
        std = K.sqrt(pred_var[:, num_classes])
        # shape: (N,)
        variance = pred_var[:, num_classes:]
        variance_depressor = K.exp(variance) - K.ones_like(variance)
        # shape: (N, C)
        pred = pred_var[:, 0:num_classes]
        # shape: (N,)
        undistorted_loss = K.categorical_crossentropy(pred, true, from_logits=True)
        # shape: (T,)
        iterable = K.variable(np.ones(T))
        dist = theano_rng.normal(avg=0, std=K.tile(std, (pred.shape[1], 1)).T, size=pred.shape)
        #distributions.Normal(loc=K.zeros_like(std), scale=std)
        monte_carlo_results = K.map_fn(gaussian_categorical_crossentropy_original(true, pred, dist), iterable, name='monte_carlo_results')

        variance_loss = K.mean(monte_carlo_results, axis=0)# * undistorted_loss
        #return K.mean(theano_rng.normal(avg=0, std=1, size=(10,)))
        # return undistorted_loss
        return variance_loss# + undistorted_loss + variance_depressor
  
    return bayesian_categorical_crossentropy_internal_original

# for a single monte carlo simulation, 
#   calculate categorical_crossentropy of 
#   predicted logit values plus gaussian 
#   noise vs true values.
# true - true values. Shape: (N, C)
# pred - predicted logit values. Shape: (N, C)
# dist - normal distribution to sample from. Shape: (N, C)
# undistorted_loss - the crossentropy loss without variance distortion. Shape: (N,)
# num_classes - the number of classes. C
# returns - total differences for all classes (N,)
def gaussian_categorical_crossentropy_original(true, pred, dist):
    def map_fn(i):
        std_samples = dist#K.transpose(dist.sample(num_classes))
        distorted_logits = pred + std_samples
        #distorted_loss = K.categorical_crossentropy(distorted_logits, true, from_logits=True)
        #return distorted_loss
        #return K.categorical_crossentropy(true, pred, from_logits=True)
        #return K.log(K.sum(K.exp(pred), axis=1))
        distorted_loss = -K.sum(distorted_logits * true, axis=1) + K.log(K.sum(K.exp(distorted_logits), axis=1))
        distorted_loss2 = K.categorical_crossentropy(distorted_logits, true, from_logits=True)
        return distorted_loss - distorted_loss2
        
    return map_fn

In [ ]:
# modified loss from https://medium.com/towards-data-science/building-a-bayesian-deep-learning-classifier-ece1845bc09
# standard categorical cross entropy
# N data points, C classes
# true - true values. Shape: (N, C)
# pred - predicted values. Shape: (N, C)
# returns - loss (N)
def categorical_cross_entropy(true, pred):
    return np.sum(true * np.log(pred), axis=1)

# Bayesian categorical cross entropy.
# N data points, C classes, T monte carlo simulations
# true - true values. Shape: (N, C)
# pred_var - predicted logit values and variance. Shape: (N, C + 1)
# returns - loss (N,)

def bayesian_categorical_crossentropy_elu(T, num_classes):
    def bayesian_categorical_crossentropy_internal_elu(true, pred_var):
        # shape: (N,)
        std = K.sqrt(pred_var[:, num_classes])
        # shape: (N,)
        variance = pred_var[:, num_classes]
        variance_depressor = K.exp(variance) - K.ones_like(variance)
        # shape: (N, C)
        pred = pred_var[:, 0:num_classes]
        # shape: (N,)
        undistorted_loss = K.categorical_crossentropy(pred, true, from_logits=True)
        # shape: (T,)
        iterable = K.variable(np.ones(T))
        dist = theano_rng.normal(avg=0, std=K.tile(std, (pred.shape[1], 1)).T, size=pred.shape)
        monte_carlo_results = K.map_fn(gaussian_categorical_crossentropy(true, pred, dist, undistorted_loss, num_classes), iterable, name='monte_carlo_results')

        variance_loss = K.mean(monte_carlo_results, axis=0) * undistorted_loss
        return variance_loss + undistorted_loss + variance_depressor
  
    return bayesian_categorical_crossentropy_internal_elu

# for a single monte carlo simulation, 
#   calculate categorical_crossentropy of 
#   predicted logit values plus gaussian 
#   noise vs true values.
# true - true values. Shape: (N, C)
# pred - predicted logit values. Shape: (N, C)
# dist - normal distribution to sample from. Shape: (N, C)
# undistorted_loss - the crossentropy loss without variance distortion. Shape: (N,)
# num_classes - the number of classes. C
# returns - total differences for all classes (N,)
def gaussian_categorical_crossentropy(true, pred, dist, undistorted_loss, num_classes):
    def map_fn(i):
        std_samples = dist
        distorted_loss = K.categorical_crossentropy(pred+dist, true, from_logits=True)
        diff = undistorted_loss - distorted_loss
        return -K.elu(diff)
    return map_fn

In [ ]:
# print(theano.printing.debugprint(model.outputs[1]))

In [ ]:
# Build model:
print('Build model...')

inp = Input(shape=(X_train.shape[1],))

xx = Dense(200,
            activation='relu',
            kernel_regularizer=l2(weight_decay),
            bias_regularizer=l2(weight_decay))(inp)

xx = Dropout(p_dropout)(xx)

xx = Dense(100,
            activation='relu',
            kernel_regularizer=l2(weight_decay),
            bias_regularizer=l2(weight_decay))(xx)
xx = Dropout(p_dropout)(xx)
logits = Dense(y_train.shape[1], 
               kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(xx)
output = Activation('softmax')(logits)
            
variance_pre = Dense(1)(xx)
variance = Activation('softplus', name='variance')(variance_pre)
logits_variance = Concatenate(name='logits_variance')([logits, variance])
#softmax_output = Activation('softmax', name='softmax_output')(logits)
    
#model = BayesianDropoutModel([inp], [softmax_output, logits_variance])


# model.compile(
#     optimizer='adam',
#     loss={
#     'logits_variance': bayesian_categorical_crossentropy_original(100, 10),
#      'softmax_output': 'categorical_crossentropy'
#     },
#     metrics={'softmax_output': keras.metrics.categorical_accuracy},
#     loss_weights={'logits_variance': 1, 
#                   'softmax_output': 0
#                 }
# )

model = BayesianDropoutModel([inp], [logits_variance])
model.compile(optimizer='adam', loss=bayesian_categorical_crossentropy_original(100, 10))

In [ ]:
# for l in model.layers:
#     print(l, l.input_shape, l.output_shape)

In [ ]:
batch_size = 200

In [ ]:
# Train model
print("Train...")

# Theano
# modeltest_1 = ModelTest(X_train[:100], 
#                         mean_y_train + std_y_train * np.atleast_2d(y_train[:100]), 
#                         test_every_X_epochs=1, verbose=0, loss='categorical', 
#                         mean_y_train=mean_y_train, std_y_train=std_y_train)
# modeltest_2 = ModelTest(X_test, 
#                         np.atleast_2d(y_test),
#                         test_every_X_epochs=1, 
#                         verbose=0, loss='categorical', 
#                         mean_y_train=mean_y_train, std_y_train=std_y_train)
model.fit(X_train,
          [y_train],
          batch_size=batch_size,
          epochs=100, 
          #callbacks=[modeltest_1, modeltest_2],
          verbose=1)



In [ ]:
# import theano
# theano.config.optimizer = 'fast_run'

In [ ]:
def score_classification(y_true, y_pred):
    return accuracy_score(y_true.argmax(-1), y_pred.argmax(-1))

In [ ]:
def score_regression(y_true, y_pred):
    raise Exception('not correct')
    return (np.mean(((mean_y_train + std_y_train * np.atleast_2d(y_true).T)
               - (y_pred + std_y_train * standard_prob))**2, 0)**0.5)

In [ ]:
standard_prob.shape

In [ ]:
# Evaluate model
# Dropout approximation for training data:
standard_prob = model.predict(X_train, batch_size=500, verbose=0)
print(score_classification(y_train, standard_prob))

# MC dropout for test data:
T = 50
prob = np.array([model.predict_stochastic(X_test, batch_size=500, verbose=0)
                 for _ in xrange(T)])
prob_mean = np.mean(prob, 0)
print(score_classification(y_test, prob_mean))

In [ ]:
# model - the trained classifier(C classes) 
#where the last layer applies softmax
# X_data - a list of input data(size N)
# T - the number of monte carlo simulations to run
def montecarlo_prediction(model, X_data, T):
# shape: (T, N, C)
    predictions = np.array([model.predict_stochastic(X_data, batch_size=750, verbose=True) for _ in range(T)])
    #print(predictions.shape)
    
    # shape: (N, C)
    prediction_probabilities = np.mean(predictions, axis=0)
    
    #print(prediction_probabilities.shape)
    # shape: (N)
    prediction_variances = np.apply_along_axis(predictive_entropy, axis=1, arr=prediction_probabilities)
    return (prediction_probabilities, prediction_variances)

# prob - prediction probability for each class(C). Shape: (N, C)
# returns - Shape: (N)
def predictive_entropy(prob):
    #print(prob.shape)
    return -1 * np.sum(np.log(prob) * prob)

In [ ]:
probs, epistemic = montecarlo_prediction(model, X_test, 50)

In [ ]:
epistemic.shape

In [ ]:
aleatoric.shape

In [ ]:
aleatoric = np.sqrt(model.predict(X_test, batch_size=500, verbose=0)[1][:, -1])

In [ ]:
from scipy.stats import mode

In [ ]:
prob_pred = prob.argmax(axis=-1)

In [ ]:
m = mode(prob_pred, 0)

In [ ]:
m.mode

In [ ]:
(prob_mean.argmax(-1) == m.mode).mean()

In [ ]:
def variation_ratio(y_pred):
    prob_pred = prob.argmax(axis=-1)
    m = mode(prob_pred, 0)
    return (1 - (m.count / float(y_pred.shape[0]))).ravel()


In [ ]:
variation_ratio(prob)

In [ ]:
# prob_std = np.std(prob, axis=0).ravel()


In [ ]:
# y_pred = mean_y_train + std_y_train * prob_mean

In [ ]:
# ind = np.argsort(y_test)
# plt.plot(y_test[ind], y_pred[ind], 'r+')

In [ ]:
p

In [ ]:
k = 25
#p=prob_std / prob_std.sum()
#vr = variation_ratio(prob)
p = epistemic / epistemic.mean() # 
p = aleatoric / aleatoric.mean()
p = p / p.sum()

fig, axs = plt.subplots(ncols=(k / 5), nrows=5)
axs = axs.ravel()
for i, i_test in enumerate(np.random.choice(len(y_test), k, p=p)):
    img = X_test[i_test].reshape(28, 28) * 255
    axs[i].imshow(img, cmap=plt.cm.get_cmap('Greys'))
    axs[i].set_title('%d - %d, ale=%.3f, epi=%.3f' % (y_test[i_test].argmax(), 
                                               prob_mean[i_test].argmax(),
                                               aleatoric[i_test],
                                               epistemic[i_test]))
fig.set_size_inches(15, 15)

In [ ]:
fig = plt.figure(111)
for yy in xrange(0, 10):
    index = y_test[:, yy].astype(bool)
    plt.scatter(epistemic[index], aleatoric[index], marker='$%d$' % yy, alpha=0.4, s=100)
fig.set_size_inches(16, 12)

plt.show()

In [ ]:
%matplotlib inline

In [ ]:
plt.hist(epistemic/epistemic.mean())
plt.hist(aleatoric/aleatoric.mean())

In [ ]:
i = 10
x0 = X_train[i]
y0 = y_train[i]
y_opt = 5

In [ ]:
plt.imshow(x0.reshape(28, 28), cmap='Greys')
plt.title('actual=%d, predicted=%d' % (y0.argmax(), model.predict(x0.reshape(1, -1))[0].argmax()))

In [ ]:
def min_f(delta_x):
    return 1 - model.predict((x0+delta_x).reshape(1, -1))[0, y_opt]

In [ ]:
delta_t = K.placeholder((None, len(x0)), dtype='float32')

In [ ]:
type(delta_t)

In [ ]:
cost = 1 - model.outputs[0][0, y_opt]

In [ ]:
f = K.function([delta_t], [cost], 
               givens={
                   K.learning_phase(): np.uint8(0),
                   model.input: delta_t + x0.astype('float32')
               })

In [ ]:
grad = K.gradients(cost, model.input)

In [ ]:
f_prime = K.function([delta_t], [grad], 
               givens={
                   K.learning_phase(): np.uint8(0),
                   model.input: delta_t + x0.astype('float32')
               })

In [ ]:
import scipy.optimize


In [ ]:
def cb(x):
    print(min_f(x))

In [ ]:
def min_f(delta):
    return f([delta.reshape(1, -1)])[0]

def min_f_prime(delta):
    return f_prime([delta.reshape(1, -1)])[0][0]

In [ ]:
res = scipy.optimize.fmin_tnc(min_f,
                              x0=np.random.normal(0, 1, size=len(x0)),
                              fprime=min_f_prime,
                              bounds=[(-0.45, 0.45)] * len(x0),
                              callback=cb)

In [ ]:
plt.imshow(res[0].reshape(28, 28), cmap='Greys', vmin=0, vmax=1)

In [ ]:
# model - the trained classifier(C classes) 
#where the last layer applies softmax
# X_data - a list of input data(size N)
# T - the number of monte carlo simulations to run
def montecarlo_prediction_epistemic_aleatoric(model, X_data, T):
# shape: (T, N, C)
    predictions = np.array([model.predict_stochastic(X_data, batch_size=750, verbose=False) for _ in range(T)])
    #print(predictions.shape)
    
    # shape: (N, C)
    prediction_probabilities = np.mean(predictions, axis=0)
    
    #print(prediction_probabilities.shape)
    # shape: (N)
    prediction_variances = np.apply_along_axis(predictive_entropy, axis=1, arr=prediction_probabilities)
    
    var = model.predict(X_data, batch_size=500)[1][:, -1]
    
    
    return (prediction_probabilities, prediction_variances, var)

# prob - prediction probability for each class(C). Shape: (N, C)
# returns - Shape: (N)
def predictive_entropy(prob):
    #print(prob.shape)
    return -1 * np.sum(np.log(prob) * prob)

In [ ]:
model.predict((x0+res[0]).reshape(1, -1))[0].argmax()

In [ ]:
prob, epi, ale = montecarlo_prediction_epistemic_aleatoric(model, (x0+res[0]).reshape(1, -1), 100)

In [ ]:
prob.argmax(), epi, ale

In [ ]:
plt.imshow((res[0] + x0).reshape(28, 28), cmap='Greys')

In [ ]:
_predict_stochastic = K.function([model.inputs[0]], [model.outputs[1]],
                                                      givens={K.learning_phase(): np.uint8(1)})

In [ ]:


o1 = np.sqrt(np.array([model._predict_loop(_predict_stochastic,
                                   [np.atleast_2d(x0)], batch_size=batch_size, verbose=False)[:, -1]
               for _ in xrange(200)]))

In [ ]:
o2 = np.sqrt(np.array([model._predict_loop(_predict_stochastic,
                                   [np.atleast_2d(x0+res[0])], batch_size=batch_size, verbose=False)[:, -1]
               for _ in xrange(200)]))

In [ ]:
o1.mean(), o1.std()

In [ ]:
o2.mean(), o2.std()

In [ ]:
probs0 = np.array([model.predict_stochastic(np.atleast_2d(x0))[0] for _ in xrange(100)])

probs0.argmax(1)

In [ ]:
probs0 = np.array([model.predict_stochastic(np.atleast_2d(x0+res[0]))[0] for _ in xrange(100)])

probs0.argmax(1)